<a href="https://colab.research.google.com/github/IA-Programming/Probando-Mistral-7B-8bit/blob/main/Mistral_7B_Instruct_Basics_8bit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1F2PeWl5FOHv4sjd7XTEu40JjqbFhC3LB?usp=sharing) [![Open In YouTube](https://img.shields.io/badge/Open%20In%20YouTube-FF0000?style=flat&logo=youtube&logoColor=white&labelColor=FF0000)](https://www.youtube.com/watch?v=OMIuP6lQXe4)


In [ ]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate xformers einops
!pip -q install langchain

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 24.

In [ ]:
!nvidia-smi

Sun Oct  1 14:05:41 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!nvidia-smi

Sun Oct  1 14:30:22 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P0    30W /  70W |   8417MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Mistral 7B Instruct



In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
torch.set_default_device('cuda')

In [ ]:


model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1",
                                                load_in_8bit=True,
                                                device_map="auto",
                                                torch_dtype="auto")

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1",
                                          torch_dtype="auto")



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
text = "<s>[INST] What is your favourite condiment? [/INST]Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!</s> [INST] Do you have mayonnaise recipes? [/INST]"

encodeds = tokenizer(text, return_tensors="pt", add_special_tokens=False)

device = 'cuda'
model_inputs = encodeds.to(device)
# model.to(device)

generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


<s>[INST] What is your favourite condiment? [/INST]Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!</s> [INST] Do you have mayonnaise recipes? [/INST] Certainly! Here is a classic recipe for mayonnaise that you can try at home:

Ingredients:

* 2 large eggs
* 1/2 cup of vegetable oil
* 1 tablespoon of white wine vinegar
* 1 tablespoon of Dijon mustard
* Salt and pepper to taste

Instructions:

1. In a medium-sized bowl, whisk together the eggs, oil, vinegar, and mustard.
2. Slowly drizzle the oil into the bowl while whisking continuously to create an emulsification.
3. Add salt and pepper to taste.
4. Refrigerate the mixture for at least an hour before serving to allow the flavours to meld.

Now, you can use this homemade mayonnaise as a base for your own variations or enjoy it on its own as a classic condiment!</s>


### Prompt Format
```
<s>[INST] <user_prompt> [/INST]

<assistant_response> </s>

[INST] <user_prompt>[/INST]
```

In [ ]:
import textwrap

def wrap_text(text, width=90): #preserve_newlines
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [ ]:
def generate(input_text, system_prompt="",max_length=512):
    prompt = f"""<s>[INST]{input_text}[/INST]"""
    inputs = tokenizer(input_text, return_tensors="pt", add_special_tokens=False)
    model_inputs = encodeds.to(device)
    # model.to(device)
    outputs = model.generate(**inputs,
                             max_length=max_length,
                             temperature=0.1,
                             do_sample=True)
    text = tokenizer.batch_decode(outputs)[0]
    wrapped_text = wrap_text(text)
    print(wrapped_text)

## CodeGen

In [ ]:
generate('''```python
def print_prime(n):
   """
   Print all primes between 1 and n
   """''', system_prompt="You are a genius python coder")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


```python
def print_prime(n):
   """
   Print all primes between 1 and n
   """
   for i in range(2, n+1):
       if is_prime(i):
           print(i)

def is_prime(n):
   """
   Check if n is prime
   """
   if n <= 1:
       return False
   for i in range(2, int(n**0.5)+1):
       if n % i == 0:
           return False
   return True

print_prime(100)
```

This code defines two functions: `print_prime` and `is_prime`. The `print_prime` function
takes an integer `n` as input and prints all primes between 1 and `n`. The `is_prime`
function takes an integer `n` as input and returns `True` if `n` is prime, and `False`
otherwise.

The `is_prime` function uses a simple algorithm to check if a number is prime. It checks
if the number is less than or equal to 1, and if so, returns `False`. It then checks if
the number is divisible by any integer between 2 and the square root of the number. If it
is divisible by any of these integers, it returns `False`. If it is not divisible by any
of these 

In [ ]:
%%time
generate('''```python
def detect_prime(n):
   """
   detect if a number is a prime number or not. return True or False
   """''')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


```python
def detect_prime(n):
   """
   detect if a number is a prime number or not. return True or False
   """
   if n <= 1:
       return False
   for i in range(2, int(n**0.5)+1):
       if n % i == 0:
           return False
   return True

def prime_numbers(n):
   """
   return a list of prime numbers less than or equal to n
   """
   prime_list = []
   for i in range(2, n+1):
       if detect_prime(i):
           prime_list.append(i)
   return prime_list

print(prime_numbers(100))</s>
CPU times: user 1min 8s, sys: 105 ms, total: 1min 8s
Wall time: 1min 9s


## Instruction Answering

In [ ]:
generate('Write a detailed analogy between mathematics and a lighthouse.',
         max_length=128)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Write a detailed analogy between mathematics and a lighthouse.

Mathematics is like a lighthouse in many ways. Just as a lighthouse guides ships safely to
harbor by emitting a steady beam of light, mathematics provides a clear and precise set of
rules and principles that guide us through the complexities of the world.

The lighthouse is a towering structure that stands tall and strong, just as mathematics is
a solid foundation upon which we can build our understanding of the world. Both are
essential for navigating through the unknown, whether it's the vast ocean or the


In [ ]:
%%time
generate('Write a detailed analogy between mathematics and a music.',
         max_length=256)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Write a detailed analogy between mathematics and a music.

Mathematics and music are two of the most fundamental and universal human disciplines.
They both involve patterns, structures, and the use of abstract concepts to understand and
create.

Mathematics can be compared to music in several ways. Just as music is composed of notes
and rhythms, mathematics is composed of numbers and operations. Both involve the use of
abstract concepts to create and understand patterns. In music, these patterns can be
simple, such as a repeating melody, or complex, such as a chord progression. Similarly, in
mathematics, patterns can range from simple arithmetic operations to complex mathematical
equations.

Both mathematics and music require a certain level of understanding and skill to create
and appreciate. In music, this skill can be developed through practice and training, such
as learning to play an instrument or compose music. In mathematics, this skill can be
developed through practice and stud

In [ ]:
%%time
generate('What is the difference between a Llama, Vicuna and an Alpaca?',
         max_length=512)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


What is the difference between a Llama, Vicuna and an Alpaca?

I'm looking for a new pet and I'm interested in these three animals. I'm not sure what the
differences are between them.

## Answer (1)

Llamas are the largest of the three, and are often used for wool. They are also used for
meat.

Vicuñas are the smallest of the three, and are often used for wool. They are also used for
meat.

Alpacas are the smallest of the three, and are often used for wool. They are also used for
meat.

The main difference between them is size.

Comment: I'm looking for a pet, not a source of food.

Comment: @user1234567890: I'm not sure what you mean by "pet". All three of these animals
are domesticated and kept as pets.

Comment: @user1234567890: I'm not sure what you mean by "pet". All three of these animals
are domesticated and kept as pets.

Comment: @user1234567890: I'm not sure what you mean by "pet". All three of these animals
are domesticated and kept as pets.

Comment: @user1234567890: I'm no

In [ ]:
%%time
generate('Write a short email to Sam Altman giving reasons to open source GPT-4',
         max_length=512)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Write a short email to Sam Altman giving reasons to open source GPT-4.

Dear Sam,

I hope this email finds you well. I am writing to you today to discuss the possibility of
open sourcing GPT-4. As you may know, GPT-4 is a highly advanced language model that has
the potential to revolutionize the field of natural language processing.

There are several reasons why I believe that open sourcing GPT-4 would be beneficial for
both OpenAI and the wider community. Firstly, by making GPT-4 available to the public, we
can encourage collaboration and innovation in the field. This could lead to new and
exciting applications for the model, and could also help to identify and fix any bugs or
limitations that may exist.

Secondly, open sourcing GPT-4 could help to build trust and transparency in the field of
AI. By making the code and models available to the public, we can demonstrate that we are
committed to being open and transparent, and that we are willing to share our knowledge
and resources wi

In [ ]:
%%time
generate('What is the capital of England?',
         max_length=256)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


What is the capital of England?

London is the capital city of England.</s>
CPU times: user 6.07 s, sys: 8.28 ms, total: 6.08 s
Wall time: 6.12 s


In [ ]:
%%time
generate('Can Geoffrey Hinton have a conversation with George Washington? Give the rationale before answering.',
         max_length=256)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Can Geoffrey Hinton have a conversation with George Washington? Give the rationale before
answering.

Geoffrey Hinton is a living person who is a computer scientist and cognitive psychologist.
George Washington was a historical figure who lived from 1732 to 1799 and was the first
President of the United States. They lived in different time periods and have no way of
communicating with each other. Therefore, it is not possible for Geoffrey Hinton to have a
conversation with George Washington.</s>
CPU times: user 44.2 s, sys: 99.7 ms, total: 44.3 s
Wall time: 44.5 s


In [ ]:
generate('Write a story about a Koala playing pool and beating all the camelids.',
         max_length=512)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Write a story about a Koala playing pool and beating all the camelids.

Once upon a time, in a land far, far away, there was a little Koala named Kiki. Kiki lived
in a tree with her family, and all day long she would play and have fun with her friends.
But one day, Kiki's friends challenged her to a game of pool. They said that they were the
best pool players in the land, and that they could beat any Koala who dared to challenge
them.

Kiki was up for the challenge. She had always loved playing pool, and she was determined
to show her friends that she was just as good as they were. So, she gathered her pool cue
and headed down to the pool hall.

When she arrived, she was surprised to see that her friends had brought along their
camelid friends to play with them. Kiki had never seen a camelid before, and she was
curious to see how they played pool.

The game began, and Kiki quickly realized that the camelids were not very good at pool.
They were clumsy and slow, and they kept knocking t

## Chat

In [ ]:
generate("""Alice: I don't know why, I'm struggling to maintain focus while studying. Any suggestion? \n\n Bob:""",
         max_length=128)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Alice: I don't know why, I'm struggling to maintain focus while studying. Any suggestion?

 Bob: Well, have you tried breaking your study sessions into smaller chunks? Maybe 25
minutes of focus followed by a 5 minute break.

 Alice: That's a good idea, I'll give it a try.

 Bob: Also, make sure you're studying in a quiet place with minimal distractions.

 Alice: I'm already studying in my room, but I'll try to minimize distractions.

 Bob: Lastly


In [ ]:
generate("""Alice: I don't know why, I'm struggling to maintain focus while studying. Any suggestion? \n\n Bob:""",
         max_length=128)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Alice: I don't know why, I'm struggling to maintain focus while studying. Any suggestion?

 Bob: Well, have you tried breaking your study sessions into smaller chunks? It's easier
to concentrate when you're studying for shorter periods of time.

 Alice: That's a good idea, I'll give it a try.

 Bob: Also, make sure you're taking breaks in between study sessions. It's important to
give your brain a break so it can recharge.

 Alice: I've been trying to take breaks, but I find myself getting


## GSM8K

In [ ]:
generate('Answer the following question by reasoning step by step. The cafeteria had 23 apples. If they used 20 for lunch, and bought 6 more, how many apple do they have?',
         max_length=256)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Answer the following question by reasoning step by step. The cafeteria had 23 apples. If
they used 20 for lunch, and bought 6 more, how many apple do they have?
Answer: 23</s>


In [ ]:
generate("Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?",
         max_length=128)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting.
How much did she earn?
A: $6</s>


In [ ]:
generate("A deep-sea monster rises from the waters once every hundred years to feast on a ship and sate its hunger. Over three hundred years, it has consumed 847 people. Ships have been built larger over time, so each new ship has twice as many people as the last ship. How many people were on the ship the monster ate in the first hundred years?",
         max_length=256)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


A deep-sea monster rises from the waters once every hundred years to feast on a ship and
sate its hunger. Over three hundred years, it has consumed 847 people. Ships have been
built larger over time, so each new ship has twice as many people as the last ship. How
many people were on the ship the monster ate in the first hundred years?
A: 14</s>
